# Inference


In [12]:
import copy
import time
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim
import torchvision.transforms as transforms
from torchsummary import summary

from utils import dataset
from models import nvidia, nvidia_2, transformer

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

WEIGHTS_FILE = "./checkpoints/nvidia_dave2.pth"
BATCH_SIZE=1
isCNN = True

def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [19]:
# For unsmoothed RMSE, execute this cell
def main():
    # Normalizing images per the paper and resizing each image to 66 x 200.
    transform = transforms.Compose([
        # Citation:
        # https://pytorch.org/vision/stable/transforms.html#scriptable-transforms
        transforms.Resize((128, 128)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))   
    ])

    # Loading in images with normalization and resizing applied.
    _ ,_, test_set = dataset.load_nvidia_dataset(transform=transform, batch_size=BATCH_SIZE)
    torch.cuda.empty_cache()

    # Loading in the NVIDIA DAVE-2 model.
    if isCNN:
        model = nvidia.NvidiaDaveCNN()
    else:
        model = transformer.VisionTransformer()
    model.load_state_dict(torch.load(WEIGHTS_FILE))
    
    
    if torch.cuda.is_available():
        model = model.to(torch.device("cuda"))

    print(summary(model, (3, 128, 128)))
    model.eval()

    cumulative_mse_losses = 0.0
    criterion = nn.MSELoss(reduction="sum")
    rmse = 0.0

    # Get the progress bar for later modification
    progress_bar = tqdm_notebook(test_set, ascii=True)

    for idx, (data, target) in enumerate(progress_bar):

        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()

        out = None
        with torch.inference_mode():
            out = model(data)
            mse = criterion(out, target)

        cumulative_mse_losses += mse

        progress_bar.set_description_str(f"Batch: {idx+1}, Loss: {(mse/len(target)):.4f}")
    rmse = (cumulative_mse_losses / len(test_set) ) ** 0.5
    print("* Unsmoothed RMSE: ", rmse.item())
    return rmse


if __name__ == '__main__':
    rmse = main()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 62, 62]           1,824
              ReLU-2           [-1, 24, 62, 62]               0
         MaxPool2d-3           [-1, 24, 61, 61]               0
       BatchNorm2d-4           [-1, 24, 61, 61]              48
            Conv2d-5           [-1, 36, 29, 29]          21,636
              ReLU-6           [-1, 36, 29, 29]               0
         MaxPool2d-7           [-1, 36, 28, 28]               0
       BatchNorm2d-8           [-1, 36, 28, 28]              72
            Conv2d-9           [-1, 48, 12, 12]          43,248
             ReLU-10           [-1, 48, 12, 12]               0
        MaxPool2d-11           [-1, 48, 11, 11]               0
      BatchNorm2d-12           [-1, 48, 11, 11]              96
           Conv2d-13             [-1, 64, 9, 9]          27,712
             ReLU-14             [-1, 6

E:\CS6601 - AI\Assignments\assignment_4\venv\lib\site-packages\ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/5614 [00:00<?, ?it/s]

* Unsmoothed RMSE:  0.1692535456470811


In [18]:
# For smoothed RMSE, execute this cell
def main():
    predictions = []
    ground_truth = []
    # Normalizing images per the paper and resizing each image to 66 x 200.
    transform = transforms.Compose([
        # Citation:
        # https://pytorch.org/vision/stable/transforms.html#scriptable-transforms
        transforms.Resize((128, 128)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))   
    ])

    # Loading in images with normalization and resizing applied.
    _ ,_, test_set = dataset.load_nvidia_dataset(transform=transform, batch_size=BATCH_SIZE)
    torch.cuda.empty_cache()

    # Loading in the NVIDIA DAVE-2 model.
    if isCNN:
        model = nvidia.NvidiaDaveCNN()
    else:
        model = transformer.VisionTransformer()
    model.load_state_dict(torch.load(WEIGHTS_FILE))
    
    
    if torch.cuda.is_available():
        model = model.to(torch.device("cuda"))

    print(summary(model, (3, 128, 128)))
    model.eval()

    cumulative_mse_losses = 0.0
    criterion = nn.MSELoss(reduction="sum")
    rmse = 0.0

    # Get the progress bar for later modification
    progress_bar = tqdm_notebook(test_set, ascii=True)

    for idx, (data, target) in enumerate(progress_bar):
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()

        out = None
        with torch.inference_mode():
            out = model(data)
            predictions.append(out.item())
            ground_truth.append(target.item())

        progress_bar.set_description_str(f"Batch: {idx+1}")
    
    # Smooth predictions and calculate RMSE
    # Reference : https://koalatea.io/python-ses-timeseries/
    alpha = 0.4
    smooth_predictions = [predictions[0]]
    for i in range(1, len(predictions)):
        predict = alpha * predictions[i-1] + (1-alpha) * smooth_predictions[i-1]
        smooth_predictions.append(predict)
        
    cumulative_mse = 0.0   
    for i, (target, prediction) in enumerate(zip(ground_truth, smooth_predictions)):
        out = criterion(torch.tensor(prediction), torch.tensor(target))
        cumulative_mse += out
        
    rmse = (cumulative_mse/len(smooth_predictions))**0.5
    print("RMSE : ", rmse)   
    return rmse


if __name__ == '__main__':
    rmse = main()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 62, 62]           1,824
              ReLU-2           [-1, 24, 62, 62]               0
         MaxPool2d-3           [-1, 24, 61, 61]               0
       BatchNorm2d-4           [-1, 24, 61, 61]              48
            Conv2d-5           [-1, 36, 29, 29]          21,636
              ReLU-6           [-1, 36, 29, 29]               0
         MaxPool2d-7           [-1, 36, 28, 28]               0
       BatchNorm2d-8           [-1, 36, 28, 28]              72
            Conv2d-9           [-1, 48, 12, 12]          43,248
             ReLU-10           [-1, 48, 12, 12]               0
        MaxPool2d-11           [-1, 48, 11, 11]               0
      BatchNorm2d-12           [-1, 48, 11, 11]              96
           Conv2d-13             [-1, 64, 9, 9]          27,712
             ReLU-14             [-1, 6

E:\CS6601 - AI\Assignments\assignment_4\venv\lib\site-packages\ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/5614 [00:00<?, ?it/s]

RMSE :  tensor(0.1614)
